In [ ]:
# import pandas as pd
# from imblearn.over_sampling import SMOTE
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import classification_report
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# df = pd.read_excel('/content/Emotions_DS_3-Emotions.xlsx')

In [ ]:
# df['Emotion'].value_counts()

,count
Emotion,
0,4098
-1,1070
1,79


In [ ]:
# # Step 1: Handle class imbalance
# # Assuming 'Emotions' is the target column
# y = df['Emotion']

In [ ]:
# # Convert 'Utterance' column to string
# df['Utterance'] = df['Utterance'].astype(str)

In [ ]:
# # Split the dataset
# X = df.drop(columns=['Emotion', 'ID'])
# y = df['Emotion']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# # Define OneHotEncoder and TF-IDF Vectorizer
# categorical_features = ['Dialogue_Act', 'Type']
# one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
# tfidf_vectorizer = TfidfVectorizer(max_features=5000)

In [ ]:
# # Column Transformer
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('tfidf', tfidf_vectorizer, 'Utterance'),
#         ('onehot', one_hot_encoder, categorical_features)
#     ])

In [ ]:
# # Step 2: Apply SMOTE with reduced k_neighbors
# smote = SMOTE(sampling_strategy='auto', k_neighbors=1, random_state=42)

In [ ]:
# # Preprocess X_train and X_test
# X_train_preprocessed = preprocessor.fit_transform(X_train)
# X_test_preprocessed = preprocessor.transform(X_test)

In [ ]:
# # Apply SMOTE to the preprocessed training data
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train_preprocessed, y_train)

In [ ]:
# # Step 3: Train Random Forest on resampled data
# rf_classifier = RandomForestClassifier(random_state=42)
# rf_classifier.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(random_state=42)

In [ ]:
# # Step 4: Evaluate the model
# y_pred = rf_classifier.predict(X_test_preprocessed)
# print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.61      0.76      0.68       212
           0       0.93      0.88      0.90       827
           1       0.00      0.00      0.00        11

    accuracy                           0.85      1050
   macro avg       0.51      0.55      0.53      1050
weighted avg       0.85      0.85      0.85      1050



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
df = pd.read_excel('/content/Emotions_DS_3-Emotions.xlsx')

In [ ]:
df['Emotion'].value_counts()

,count
Emotion,
0,4098
-1,1070
1,79


In [ ]:
# Convert 'Utterance' column to string
df['Utterance'] = df['Utterance'].astype(str)

In [ ]:
# Step 1: Handle class imbalance (target column is 'Emotions')
y = df['Emotion']

In [ ]:
# Split the dataset into features and target
X = df.drop(columns=['Emotion', 'ID'])
y = df['Emotion']

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Step 2: Define the preprocessing pipeline
categorical_features = ['Dialogue_Act', 'Type']
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')

In [ ]:
# Define Column Transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('tfidf', TfidfVectorizer(max_features=5000), 'Utterance'),  # TF-IDF for 'Utterance'
        ('onehot', one_hot_encoder, categorical_features)           # One-hot encoding for categorical columns
    ])

In [ ]:
# Step 3: Define the class weights
# Automatically calculate class weights based on the class distribution
class_weights = compute_class_weight(class_weight='balanced', classes=pd.unique(y_train), y=y_train)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

In [ ]:
# 1. Get unique classes in y_train
unique_classes = y_train.unique()

In [ ]:
# 2. Update class_weight_dict with any missing classes
for cls in unique_classes:
    if cls not in class_weight_dict:
        class_weight_dict[cls] = 1

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42, class_weight=class_weight_dict)

In [ ]:
# Step 5: Set up SMOTE for handling class imbalance, and define the pipeline
# Get class counts for the sampling strategy
class_counts = y_train.value_counts()
sampling_strategy = {cls: int(class_counts[cls] * 2) for cls in class_counts.index if class_counts[cls] > 1}

In [ ]:
# SMOTE with limited neighbors to avoid errors on small classes
smote = SMOTE(sampling_strategy=sampling_strategy, k_neighbors=1, random_state=42)

In [ ]:
# Step 6: Define the complete pipeline (preprocessing, SMOTE, and classifier)
pipeline = ImbPipeline(steps=[
    ('preprocessor', preprocessor),  # Preprocess (TF-IDF + OneHotEncoding)
    ('smote', smote),                # SMOTE for handling class imbalance
    ('classifier', rf_classifier)    # Random Forest Classifier
])

In [ ]:
# Step 7: Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('tfidf',
                                                  TfidfVectorizer(max_features=5000),
                                                  'Utterance'),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Dialogue_Act', 'Type'])])),
                ('smote',
                 SMOTE(k_neighbors=1, random_state=42,
                       sampling_strategy={-1: 1716, 0: 6542, 1: 136})),
                ('classifier',
                 RandomForestClassifier(class_weight={-1: 1,
                                                      0: 0.4276979516967288,
                                                      1: 1.6305361305361306,
                                                      2: 20.573529411764707},
                                        random_state=42))])

In [ ]:
# Step 8: Predict and evaluate the model
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.68      0.63      0.65       212
           0       0.90      0.92      0.91       827
           1       0.00      0.00      0.00        11

    accuracy                           0.85      1050
   macro avg       0.52      0.52      0.52      1050
weighted avg       0.84      0.85      0.85      1050



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
